In [4]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np


In [5]:
## CREATE GLOBE MAP 

# fig = go.Figure(go.Scattergeo())
# fig.update_geos(projection_type="orthographic")
# fig.update_layout(height=300, margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

In [6]:
data_hoja1 = pd.read_excel('CO2.xlsx', sheet_name='fossil_CO2_totals_by_country')


In [7]:
data_h1 = data_hoja1.melt(
    id_vars=['ISOcode', 'Country'],  # fixed columns
    var_name='Year',                 # new col
    value_name='CO2 Emissions'       # ew col 
)

In [9]:
df_year = data_h1[data_h1['Year'] == 2020]

In [10]:
df_year

,ISOcode,Country,Year,CO2 Emissions
10450,ABW,Aruba,2020,1.144333
10451,AFG,Afghanistan,2020,8.183718
10452,AGO,Angola,2020,23.055844
10453,AIA,Anguilla,2020,0.022779
10454,ALB,Albania,2020,4.706920
...,...,...,...,...
10654,WSM,Samoa,2020,0.347398
10655,YEM,Yemen,2020,11.607959
10656,ZAF,South Africa,2020,427.815947
10657,ZMB,Zambia,2020,6.618359


In [13]:
## PUNTOS POR CADA PAIS 

fig = go.Figure(go.Scattergeo(
    locationmode='country names',
    locations=df_year['Country'],
    text=df_year['CO2 Emissions'],
    marker=dict(
        size=10,
        color=df_year['CO2 Emissions'],
        colorscale='Viridis',
        colorbar=dict(title="CO2 Emissions")
    )
))

fig.update_geos(projection_type="orthographic")
fig.update_layout(
    title=f'CO2 Emissions by Country in {2020}',
    height=600,
    margin={"r":0,"t":0,"l":0,"b":0}
)

fig.show()

In [14]:
fig = go.Figure(data=go.Choropleth(
    locations=df_year['Country'],
    locationmode='country names',
    z=df_year['CO2 Emissions'],
    text=df_year['CO2 Emissions'],
    colorscale='Viridis',
    colorbar=dict(title="CO2 Emissions")
))

fig.update_geos(projection_type="orthographic")
fig.update_layout(
    title=f'CO2 Emissions by Country in {2020}',
    height=600,
    margin={"r":0,"t":0,"l":0,"b":0}
)

fig.show()

______________________________________________________________

In [59]:
data_hoja3 = pd.read_excel('CO2.xlsx', sheet_name='fossil_CO2_by_sector_and_countr')


In [60]:
# Transformar los datos para tener 'Year' y 'CO2 Emissions' como columnas
data_3 = data_hoja3.melt(
    id_vars=['Sector', 'ISOcode', 'Country'],  # columnas fijas
    var_name='Year',                 # nueva columna
    value_name='CO2 Emissions'       # nueva columna 
)

In [61]:
# Convertir la columna 'Year' a tipo numérico
data_3['Year'] = pd.to_numeric(data_3['Year'])

# Crear una nueva columna para las décadas
data_3['Decade'] = (data_3['Year'] // 10) * 10


In [62]:
grouped_data = data_3.groupby(['Sector', 'Decade', 'Country'])['CO2 Emissions'].mean().reset_index()


In [63]:
import pandas as pd
from dash import Dash, dcc, html, callback, Output, Input
import plotly.express as px


# # Cargar la hoja de datos
# data_hoja3 = pd.read_excel('CO2.xlsx', sheet_name='fossil_CO2_totals_by_country')

# # Transformar los datos para tener 'Year' y 'CO2 Emissions' como columnas
# data_3 = data_hoja3.melt(
#     id_vars=['Sector', 'ISOcode', 'Country'],  # columnas fijas
#     var_name='Year',                 # nueva columna
#     value_name='CO2 Emissions'       # nueva columna 
# )

# # Convertir la columna 'Year' a tipo numérico
# data_3['Year'] = pd.to_numeric(data_3['Year'])

# # Crear una nueva columna para las décadas
# data_3['Decade'] = (data_3['Year'] // 10) * 10

# Agrupar por 'Sector', 'Decade' y 'Country' y calcular la media de 'CO2 Emissions'
# grouped_data = data_3.groupby(['Sector', 'Decade', 'Country'])['CO2 Emissions'].mean().reset_index()

# Crear la aplicación Dash
app = Dash(__name__)

# Layout de la aplicación
app.layout = html.Div([
    dcc.Checklist(
        id='checklist-sectors',
        options=[{'label': sector, 'value': sector} for sector in grouped_data['Sector'].unique()],
        value=[grouped_data['Sector'].unique()[0]],
        labelStyle={'display': 'block'}
    ),
    dcc.Dropdown(
        id='dropdown-decade',
        options=[{'label': str(decade), 'value': decade} for decade in grouped_data['Decade'].unique()],
        value=grouped_data['Decade'].unique()[0],
        placeholder="Select a decade"
    ),
    dcc.Graph(id='bar-chart')
])

# Callback para actualizar el gráfico de barras
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('checklist-sectors', 'value'),
     Input('dropdown-decade', 'value')]
)
def update_bar_chart(selected_sectors, selected_decade):
    filtered_data = grouped_data[(grouped_data['Sector'].isin(selected_sectors)) & (grouped_data['Decade'] == selected_decade)]
    
    fig = px.bar(filtered_data, x='Country', y='CO2 Emissions', color='Sector', title=f'CO2 Emissions by Country in {selected_decade}s')
    
    return fig

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)